Current number of unique href links: 45
Current number of unique href links: 90
Current number of unique href links: 135
Number of unique href links: 135
Review section not found for index: 3
Review section not found for index: 4
Review section not found for index: 5


KeyboardInterrupt: 

Review section not found for index: 3


KeyboardInterrupt: 

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

def get_href_links(driver, wait, actions, num_items_to_fetch=100):
    href_links = set()
    while len(href_links) < num_items_to_fetch:
        actions.send_keys(Keys.END)
        time.sleep(2)
        
        for x in range(1, 46):
            try:
                xpath = f'//*[@id="root"]/main/div/section[3]/div[1]/div/div[{x}]/div/div[2]/a[2]'
                element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
                href = element.get_attribute('href')
                href_links.add(href)
            except (NoSuchElementException, TimeoutException):
                continue

        print(f"Current number of unique href links: {len(href_links)}")
        if len(href_links) >= num_items_to_fetch:
            break

    return list(href_links)

def extract_reviews(driver, wait):
    actions = driver.find_element(By.CSS_SELECTOR, 'body')
    time.sleep(1)
    actions.send_keys(Keys.END)
    time.sleep(2)
    actions.send_keys(Keys.END)
    time.sleep(2)
    actions.send_keys(Keys.END)
    time.sleep(1)

    
    reviews = []
    for n in range(3, 10):
        try:
            review_id = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[1]/div/div[1]/p[1]'))).text
            print(f"review_id: {review_id}")
            #//*[@id="style_estimate_list"]/div/div[3]/div[1]/div/div[1]/p[1]
            try:
                weight_height_gender = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[1]/div/div[2]/p[1]'))).text
                print(f"weight_height_gender: {weight_height_gender}")
            except:
                weight_height_gender = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[1]/div/div[2]/p'))).text
                print(f"weight_height_gender (alternative): {weight_height_gender}")
            
            
            top_size = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[1]/ul/li[1]/span'))).text
            print(f"top_size: {top_size}")
            
            brightness_comment = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[1]/ul/li[2]/span'))).text
            print(f"brightness_comment: {brightness_comment}")
            
            color_comment = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[1]/ul/li[3]/span'))).text
            print(f"color_comment: {color_comment}")
            
            thickness_comment = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[1]/ul/li[4]/span'))).text
            print(f"thickness_comment: {thickness_comment}")
            
            purchased_product_id = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[2]/div[2]/a'))).get_attribute('href')
            print(f"purchased_product_id: {purchased_product_id}")
            
            purchased_size = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[2]/div[2]/p/span'))).text
            print(f"purchased_size: {purchased_size}")
            
            comment = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[2]'))).text
            print(f"comment: {comment}")

            review = {
                "weight_height_gender": weight_height_gender,
                "review_id": review_id,
                "top_size": top_size,
                "brightness_comment": brightness_comment,
                "color_comment": color_comment,
                "thickness_comment": thickness_comment,
                "purchased_product_id": purchased_product_id,
                "purchased_size": purchased_size,
                "comment": comment
            }
            reviews.append(review)
        except (NoSuchElementException, TimeoutException):
            print(f"Review information not found for element index: {n}")
            continue
    
    return reviews

def get_product_info(driver, wait, href_links):
    products = []
    for index, link in enumerate(href_links):
        driver.get(link)
        time.sleep(2)  # 페이지 로드 대기

        try:
            product_id = f"top{index + 1}"
            
            try:
                product_name = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div[3]/h2'))).text
            except (NoSuchElementException, TimeoutException):
                product_name = "N/A"
                print(f"Product name not found for link: {link}")
                
            try:
                category = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div[2]/a[1]'))).text
            except (NoSuchElementException, TimeoutException):
                category = "N/A"
                print(f"Category not found for link: {link}")

            try:
                price = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div[5]/div/div/span'))).text.strip()
            except (NoSuchElementException, TimeoutException):
                price = "N/A"
                print(f"Price not found for link: {link}")

            try:
                image_element = wait.until(EC.presence_of_element_located((By.XPATH, '//img[@class="sc-1jl6n79-4 AqRjD"]')))
                image_url = image_element.get_attribute('src')
            except (NoSuchElementException, TimeoutException):
                image_url = "N/A"
                print(f"Image URL not found for link: {link}")

            description = link

            # 사이즈 정보 가져오기
            sizes = []
            try:
                ul_element = wait.until(EC.presence_of_element_located((By.XPATH, '//ul[contains(@class, "sc-1sxlp32-1") or contains(@class, "sc-8wsa6t-1 Qtsoe")]')))
                li_elements = ul_element.find_elements(By.TAG_NAME, 'li')
                for li in li_elements:
                    size_text = li.text.strip()
                    if size_text not in ["cm", "MY"]:
                        sizes.append(size_text)
            except (NoSuchElementException, TimeoutException):
                print(f"Size information not found for link: {link}")

            # 리뷰 정보 가져오기
            reviews = extract_reviews(driver, wait)

            product = {
                "product_id": product_id,
                "product_name": product_name,
                "category": category,
                "price": price,
                "image_url": image_url,
                "description": description,
                "size": sizes,
                "reviews": reviews
            }
            products.append(product)
            
            # 데이터 출력
            print(product)

        except Exception as e:
            print(f"Failed to extract data for link: {link} due to {e}")
            continue
    
    return products

def save_to_csv(products, filename="products.csv"):
    df = pd.DataFrame(products)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"Data saved to {filename}")

def main():
    URL = "https://www.musinsa.com/categories/item/001?device=mw&sortCode=emt_high"
    option = webdriver.ChromeOptions()
    option.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=option)
    driver.get(URL)
    time.sleep(3)

    driver.find_element(By.XPATH, '//*[@id="root"]/main/div/div[3]/div/button').click()
    time.sleep(1)

    wait = WebDriverWait(driver, 10)
    actions = driver.find_element(By.CSS_SELECTOR, 'body')

    href_links = get_href_links(driver, wait, actions, num_items_to_fetch=100)
    print(f"Number of unique href links: {len(href_links)}")

    products = get_product_info(driver, wait, href_links)
    print("Products extracted:")
    for product in products:
        print(product)

    save_to_csv(products)

    # driver.quit()

if __name__ == '__main__':
    main()


Current number of unique href links: 45
Current number of unique href links: 90
Current number of unique href links: 135
Number of unique href links: 135
review_id: LV.3 덩민덩민
weight_height_gender: 여성 · 162cm · 55kg
top_size: 보통이에요
brightness_comment: 보통이에요
color_comment: 보통이에요
thickness_comment: 얇아요
purchased_product_id: https://www.musinsa.com/app/goods/2402005/0
purchased_size: 화이트/S
comment: 엄청 얇아요... 안에가 다 비쳐요...
친구가 로고보고 시진핑이냐고 물어봤어요...
그래도 예뻐요 :)
review_id: LV.3 슈퍼킹카우티비
Review information not found for element index: 4
review_id: LV.4 개구리폴쨕
weight_height_gender: 여성 · 162cm · 42kg
top_size: 보통이에요
brightness_comment: 보통이에요
color_comment: 보통이에요
thickness_comment: 보통이에요
purchased_product_id: https://www.musinsa.com/app/goods/2402005/0
purchased_size: 화이트/S
comment: 사랑하는 사람 따라 구매했서용🐱💜 생각했던것 보단 작았는데 그래도 만족스러움!! 비침도 거의 없고 레이어드해서 입기에두 조아요 넘 예쁘당 낭낭하게 입으시려면 m으로 사세요!!
review_id: LV.5 소 잃고 뇌약간 고치기
Review information not found for element index: 6
review_id: LV.6 ssssg
weight_height_gender: 여

In [ ]:
# 테스트 코드
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

def get_href_links(driver, wait, actions, num_items_to_fetch=100):
    href_links = set()
    while len(href_links) < num_items_to_fetch:
        actions.send_keys(Keys.END)
        time.sleep(2)
        
        for x in range(1, 46):
            try:
                xpath = f'//*[@id="root"]/main/div/section[3]/div[1]/div/div[{x}]/div/div[2]/a[2]'
                element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
                href = element.get_attribute('href')
                href_links.add(href)
            except (NoSuchElementException, TimeoutException):
                continue

        print(f"Current number of unique href links: {len(href_links)}")
        if len(href_links) >= num_items_to_fetch:
            break

    return list(href_links)

def extract_reviews(driver, wait):
    actions = driver.find_element(By.CSS_SELECTOR, 'body')
    time.sleep(1)
    actions.send_keys(Keys.END)
    time.sleep(2)
    actions.send_keys(Keys.END)
    time.sleep(2)
    actions.send_keys(Keys.END)
    time.sleep(1)

    
    reviews = []
    for n in range(3, 10):
        try:
            review_id = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[1]/div/div[1]/p[1]'))).text
            print(f"review_id: {review_id}")
            #//*[@id="style_estimate_list"]/div/div[3]/div[1]/div/div[1]/p[1]
            try:
                weight_height_gender = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[1]/div/div[2]/p[1]'))).text
                print(f"weight_height_gender: {weight_height_gender}")
            except:
                weight_height_gender = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[1]/div/div[2]/p'))).text
                print(f"weight_height_gender (alternative): {weight_height_gender}")
            
            
            top_size = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[1]/ul/li[1]/span'))).text
            print(f"top_size: {top_size}")
            
            brightness_comment = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[1]/ul/li[2]/span'))).text
            print(f"brightness_comment: {brightness_comment}")
            
            color_comment = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[1]/ul/li[3]/span'))).text
            print(f"color_comment: {color_comment}")
            
            thickness_comment = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[1]/ul/li[4]/span'))).text
            print(f"thickness_comment: {thickness_comment}")
            
            purchased_product_id = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[2]/div[2]/a'))).get_attribute('href')
            print(f"purchased_product_id: {purchased_product_id}")
            
            purchased_size = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[2]/div[2]/p/span'))).text
            print(f"purchased_size: {purchased_size}")
            
            comment = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[2]'))).text
            print(f"comment: {comment}")

            review = {
                "weight_height_gender": weight_height_gender,
                "review_id": review_id,
                "top_size": top_size,
                "brightness_comment": brightness_comment,
                "color_comment": color_comment,
                "thickness_comment": thickness_comment,
                "purchased_product_id": purchased_product_id,
                "purchased_size": purchased_size,
                "comment": comment
            }
            reviews.append(review)
        except (NoSuchElementException, TimeoutException):
            print(f"Review information not found for element index: {n}")
            continue
    
    return reviews

def get_product_info(driver, wait, href_links):
    products = []
    for index, link in enumerate(href_links[63:], start=64):  # 64번째 링크부터 시작
        driver.get(link)
        time.sleep(2)  # 페이지 로드 대기

        try:
            product_id = f"top{index + 1}"
            
            try:
                product_name = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div[3]/h2'))).text
            except (NoSuchElementException, TimeoutException):
                product_name = "N/A"
                print(f"Product name not found for link: {link}")
                
            try:
                category = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div[2]/a[1]'))).text
            except (NoSuchElementException, TimeoutException):
                category = "N/A"
                print(f"Category not found for link: {link}")

            try:
                price = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div[5]/div/div/span'))).text.strip()
            except (NoSuchElementException, TimeoutException):
                price = "N/A"
                print(f"Price not found for link: {link}")

            try:
                image_element = wait.until(EC.presence_of_element_located((By.XPATH, '//img[@class="sc-1jl6n79-4 AqRjD"]')))
                image_url = image_element.get_attribute('src')
            except (NoSuchElementException, TimeoutException):
                image_url = "N/A"
                print(f"Image URL not found for link: {link}")

            description = link

            # 사이즈 정보 가져오기
            sizes = []
            try:
                ul_element = wait.until(EC.presence_of_element_located((By.XPATH, '//ul[contains(@class, "sc-1sxlp32-1") or contains(@class, "sc-8wsa6t-1 Qtsoe")]')))
                li_elements = ul_element.find_elements(By.TAG_NAME, 'li')
                for li in li_elements:
                    size_text = li.text.strip()
                    if size_text not in ["cm", "MY"]:
                        sizes.append(size_text)
            except (NoSuchElementException, TimeoutException):
                print(f"Size information not found for link: {link}")

            # 리뷰 정보 가져오기
            reviews = extract_reviews(driver, wait)

            product = {
                "product_id": product_id,
                "product_name": product_name,
                "category": category,
                "price": price,
                "image_url": image_url,
                "description": description,
                "size": sizes,
                "reviews": reviews
            }
            products.append(product)
            
            # 데이터 출력
            print(product)

        except Exception as e:
            print(f"Failed to extract data for link: {link} due to {e}")
            continue
    
    return products

def save_to_csv(products, filename="product_test.csv"):
    df = pd.DataFrame(products)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"Data saved to {filename}")

def main():
    URL = "https://www.musinsa.com/categories/item/001?device=mw&sortCode=emt_high"
    option = webdriver.ChromeOptions()
    option.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=option)
    driver.get(URL)
    time.sleep(3)

    driver.find_element(By.XPATH, '//*[@id="root"]/main/div/div[3]/div/button').click()
    time.sleep(1)

    wait = WebDriverWait(driver, 10)
    actions = driver.find_element(By.CSS_SELECTOR, 'body')

    href_links = get_href_links(driver, wait, actions, num_items_to_fetch=100)
    print(f"Number of unique href links: {len(href_links)}")

    products = get_product_info(driver, wait, href_links)
    print("Products extracted:")
    for product in products:
        print(product)

    save_to_csv(products, filename="product_test.csv")

    # driver.quit()

if __name__ == '__main__':
    main()
